<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/others/rapid_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## numba での高速化

### njit: numba の nopython モード

[numpy での重い計算の例](https://qiita.com/gyu-don/items/9d223b007ca620e95abc)

In [43]:
import sys
sys.setrecursionlimit(100000)

def ack(m, n):
    if m == 0:
        return n + 1
    if n == 0:
        return ack(m - 1, 1)
    return ack(m - 1, ack(m, n - 1))

通常時の計算時間を測定

In [44]:
import time
from contextlib import contextmanager

@contextmanager
def timer(target: str):
    t = time.perf_counter()
    yield None
    # 有効数字3桁表示
    print(f"[{target}]: {time.perf_counter() - t: .3g} s")

with timer("ack(3, 10)"):
    ack(3, 10)

[ack(3, 10)]:  8.89 s


njit で高速化。[型推論は自動。ポリモーフィズムも対応。](https://numba.readthedocs.io/en/stable/user/jit.html)

In [45]:
from numba import njit

with timer("define njit function"):
    @njit(cache=True)
    def lazy_ack(m, n):
        if m == 0:
            return n + 1
        if n == 0:
            return lazy_ack(m - 1, 1)
        return lazy_ack(m - 1, lazy_ack(m, n - 1))

# コンパイル時間含む
with timer("1st try"):
    lazy_ack(3, 10)

# コンパイル時間含まない
with timer("2nd try"):
    lazy_ack(3, 10)

[define njit function]:  0.00184 s
[1st try]:  0.615 s
[2nd try]:  0.239 s


定義時に型指定でコンパイル。トータル時間は変化なし。
ポリモーフィズムもないため処理速度を図る目的等がなければ使わなくてもよい。

In [46]:
with timer("define njit function"):
    @njit("int32(int32, int32)", cache=True)
    def eager_ack(m, n):
        if m == 0:
            return n + 1
        if n == 0:
            return eager_ack(m - 1, 1)
        return eager_ack(m - 1, eager_ack(m, n - 1))

# 初回
with timer("compile time included"):
    eager_ack(3, 10)

# 事前にコンパイル済なので二回目以降の計算時間も同等
with timer("compile time excluded"):
    eager_ack(3, 10)

[define njit function]:  0.103 s
[compile time included]:  0.236 s
[compile time excluded]:  0.241 s


### 並列化 & fastmath


[検証が容易な2重ループの例](https://qiita.com/AnchorBlues/items/59a8543765549fe7bac0)
\begin{equation}
 S_L=\sum_{i=0}^{L-1}\sum_{j=0}^{L-1} (i-j)
\end{equation}

In [47]:
def double_sum(size):
    sum = 0
    for i in range(size):
        for j in range(size):
            sum += i - j
    return sum

size = 10000

with timer("pure python"):
    # 0 となることを確認
    print(f"ans={double_sum(size)}")

ans=0
[pure python]:  8.33 s


numba を用いたケース

In [48]:
with timer("define njit function"):
    @njit("int32(int32)", cache=True)
    def double_sum_njit(size):
        sum = 0
        for i in range(size):
            for j in range(size):
                sum += i - j
        return sum

with timer("njit enabled"):
    print(f"ans={double_sum_njit(size)}")

[define njit function]:  0.0729 s
ans=0
[njit enabled]:  3.74e-05 s


更に高速化するには[追加オプション](https://numba.readthedocs.io/en/stable/user/performance-tips.html)を使用する。

次の例は prange を用いた並列化. ただしこのケースではマルチコア化のバウンドの方が大きいため高速化されない。

In [49]:
from numba import prange

with timer("define njit function"):
    @njit("int32(int32)", cache=True, parallel=True)
    def double_sum_parallel(size):
        sum = 0
        # 並列化の対象のループには prange を使う
        for i in prange(size):
            for j in range(size):
                sum += i - j
        return sum

with timer("njit with parallel"):
    print(f"ans={double_sum_parallel(size)}")

[define njit function]:  0.32 s
ans=0
[njit with parallel]:  0.00307 s


C++ の最適化のように fastmath も使用可能

In [50]:
with timer("define njit function"):
    @njit("int32(int32)", cache=True, parallel=True, fastmath=True)
    def double_sum_fast(size):
        sum = 0
        for i in prange(size):
            for j in range(size):
                sum += i - j
        return sum

with timer("njit with parallel & fastmath"):
    print(f"ans={double_sum_fast(size)}")

[define njit function]:  0.458 s
ans=0
[njit with parallel & fastmath]:  0.00137 s


### numba.cuda

[GPUを意識したプログラミング](https://co-crea.jp/wp-content/uploads/2016/07/File_2.pdf)が必要：
- [グリッド・ブロック中のスレッド位置の取得方法](https://numba.pydata.org/numba-doc/latest/cuda/kernels.html#absolute-positions)
- [CPU・GPU間のデータ転送方法](https://numba.pydata.org/numba-doc/latest/cuda/memory.html)

In [51]:
from numba import cuda
import numpy as np
import sys
sys.setrecursionlimit(100000)


# カーネル関数
@cuda.jit
def add_kernel(a, b, c):
    i = cuda.grid(1)
    c[i] = a[i] + b[i]

# 起動関数
def add_arrays(a, b, threads_per_block=256):
    # threads_per_block は1ブロックあたりのスレッド数 (128~512)
    # GPU の使用ブロック数を計算
    blocks = (a.size + threads_per_block - 1) // threads_per_block

    # 結果保存用にメモリ確保
    result = cuda.to_device(np.zeros_like(a))
    add_kernel[blocks, threads_per_block](
        cuda.to_device(a), cuda.to_device(b), result)
    return result.copy_to_host()

array_size = 100000000
a = np.ones(array_size, dtype=np.float32)
b = np.ones(array_size, dtype=np.float32)

with timer("CPU computation"):
    a + b

add_arrays(a, b)
with timer("GPU computation"):
    add_arrays(a, b)

[CPU computation]:  0.171 s
[GPU computation]:  0.568 s


## concurrent.futures での高速化

### オリジナルの API

CPU 数を確認

In [52]:
import os

print(f"CPU count is {os.cpu_count()}.")
print(f"CPU for current thread is {len(os.sched_getaffinity(0))}.")

import multiprocessing
print(f"CPU count by multiprocessing is {multiprocessing.cpu_count()}.")

CPU count is 2.
CPU for current thread is 2.
CPU count by multiprocessing is 2.


ProcessPoolExecutor は CPU bound のタスクに有効。

以下はmax_workers の最適値を見つけるサンプル。

In [53]:
import numpy as np
import concurrent.futures

def do_something(size):
    return np.dot(np.ones((size, size)), np.ones((size, size)))

worker_values = [1, 2, 4, 8, 16, 32, 64]
tasks = [1000] * 10

for max_workers in worker_values:
    with timer(f"max_workers={max_workers}"):
        with concurrent.futures.ProcessPoolExecutor(max_workers) as executor:
            futures = [executor.submit(do_something, task) for task in tasks]
            results = [f.result() for f in concurrent.futures.as_completed(futures)]

[max_workers=1]:  1.15 s
[max_workers=2]:  0.854 s
[max_workers=4]:  0.9 s
[max_workers=8]:  1.04 s
[max_workers=16]:  1.28 s
[max_workers=32]:  1.79 s
[max_workers=64]:  2.75 s


上記は submit メソッドを使用した。
submit は for 文などで順次タスクを追加する場合に便利。

既に処理対象がリストなどでまとまっている場合は map メソッドが便利。使い方は以下。

In [54]:
import numpy as np
import concurrent.futures

def do_something(size):
    return np.dot(np.ones((size, size)), np.ones((size, size)))

worker_values = [1, 2, 4, 8, 16, 32, 64]
tasks = [1000] * 10

for max_workers in worker_values:
    with timer(f"max_workers={max_workers}"):
        with concurrent.futures.ProcessPoolExecutor(max_workers) as executor:
            results = executor.map(do_something, tasks)

[max_workers=1]:  0.778 s
[max_workers=2]:  1.05 s
[max_workers=4]:  1.36 s
[max_workers=8]:  1.62 s
[max_workers=16]:  1.47 s
[max_workers=32]:  2 s
[max_workers=64]:  3.99 s


ThreadPoolExecutor は I/O bound のタスクに有効

In [55]:
import numpy as np
import requests
import concurrent.futures

def do_something(url):
    response = requests.get(url)
    return response

worker_values = [1, 2, 4, 8, 16, 32, 64]
tasks = ['https://google.com', 'https://facebook.com', 'https://twitter.com',
         'https://www.youtube.com/', 'https://www.amazon.com/',
         'https://github.com/']

for max_workers in worker_values:
    with timer(f"max_workers={max_workers}"):
        with concurrent.futures.ThreadPoolExecutor(max_workers) as executor:
            results = executor.map(do_something, tasks)

[max_workers=1]:  2.04 s
[max_workers=2]:  1.05 s
[max_workers=4]:  0.483 s
[max_workers=8]:  0.937 s
[max_workers=16]:  0.442 s
[max_workers=32]:  0.444 s
[max_workers=64]:  0.451 s


結果はジェネレータ式なので for 文で処理可能

In [56]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = executor.map(do_something, tasks)

print(f"type_of_results={type(results)}")
for task, result in zip(tasks, results):
    print(f"{task}: code {result.status_code}")

type_of_results=<class 'generator'>
https://google.com: code 200
https://facebook.com: code 200
https://twitter.com: code 200
https://www.youtube.com/: code 200
https://www.amazon.com/: code 503
https://github.com/: code 200


### tqdm.contrib の利用

tqdm.contrib.concurrent.thread_map で tqdm と併用可能。表記もシンプル。

In [57]:
from tqdm.contrib.concurrent import thread_map


def do_something(url):
    response = requests.get(url)
    return response

worker_values = [1, 2, 4, 8, 16, 32, 64]
tasks = ['https://google.com', 'https://facebook.com', 'https://twitter.com',
         'https://www.youtube.com/', 'https://www.amazon.com/',
         'https://github.com/']

results = list(thread_map(do_something, tasks))

  0%|          | 0/6 [00:00<?, ?it/s]

tqdm.contrib.concurrent.process_map も同様

In [58]:
from tqdm.contrib.concurrent import process_map

def do_something(size):
    return np.dot(np.ones((size, size)), np.ones((size, size)))

tasks = [1000] * 10
results = list(process_map(do_something, tasks))

  0%|          | 0/10 [00:00<?, ?it/s]

## データロードの高速化

In [59]:
%%capture
!pip install dask[dataframe] orjson

In [60]:
import pandas as pd
import dask.dataframe as dd
import json
import orjson

In [61]:
import urllib

url = "https://github.com/mwaskom/seaborn-data/raw/refs/heads/master/diamonds.csv"

with urllib.request.urlopen(url) as web_file:
  with open("diamonds.csv", 'wb') as local_file:
    local_file.write(web_file.read())

In [62]:
%%time
df = pd.read_csv("data-large.csv")

CPU times: user 819 ms, sys: 126 ms, total: 945 ms
Wall time: 975 ms


<timed exec>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.


In [63]:
%%time
df = dd.read_csv("data-large.csv").compute()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


CPU times: user 4.17 s, sys: 297 ms, total: 4.46 s
Wall time: 7.51 s


In [64]:
url = "https://github.com/json-iterator/test-data/raw/refs/heads/master/large-file.json"

with urllib.request.urlopen(url) as web_file:
  with open("large-file.json", 'wb') as local_file:
    local_file.write(web_file.read())

In [65]:
%%time
with open("large-file.json") as f:
    data = json.load(f)

CPU times: user 505 ms, sys: 117 ms, total: 622 ms
Wall time: 624 ms


In [66]:
%%time
with open("large-file.json") as f:
    data = orjson.loads(f.read())

CPU times: user 185 ms, sys: 180 ms, total: 365 ms
Wall time: 367 ms
